# Router LLM Serving Agent - 커피 키오스크 주문 시스템

## 1. 환경 설정 및 라이브러리 임포트

In [1]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수를 로드한다
load_dotenv()

True

In [ ]:
# 필요한 라이브러리 설치
# !pip install openai pydantic requests

In [2]:
import os
import json
import time
import requests
from typing import List, Dict, Optional, Literal, Any
from datetime import datetime
from openai import OpenAI
from pydantic import BaseModel, Field, validator
from enum import Enum

# OpenAI 클라이언트 초기화
client = OpenAI()

## 2. Pydantic 모델 정의

In [3]:
class ServingType(str, Enum):
    """LLM 서빙 타입 열거형"""
    CLOUD = "cloud"
    LOCAL = "local"


class RequestCharacteristics(BaseModel):
    """요청의 특성을 분석한 결과를 표현하는 모델"""
    complexity: Literal["simple", "moderate", "complex"] = Field(
        description="질문의 복잡도"
    )
    privacy_sensitivity: Literal["low", "medium", "high"] = Field(
        description="프라이버시 민감도"
    )
    response_time_requirement: Literal["fast", "balanced", "quality"] = Field(
        description="응답 시간 요구사항"
    )
    contains_sensitive_data: bool = Field(
        description="민감 정보 포함 여부"
    )
    estimated_tokens: int = Field(
        ge=0,
        description="예상 토큰 수"
    )
    reasoning: str = Field(
        description="분석 근거"
    )


class ServingEndpoint(BaseModel):
    """LLM 서빙 엔드포인트 정보를 표현하는 모델"""
    serving_type: ServingType = Field(description="서빙 타입")
    endpoint_name: str = Field(description="엔드포인트 이름")
    model_name: str = Field(description="모델 이름")
    is_available: bool = Field(default=True, description="서비스 가용 여부")
    avg_latency_ms: int = Field(ge=0, description="평균 지연시간 (밀리초)")
    cost_per_1k_tokens: float = Field(ge=0, description="1K 토큰당 비용 (USD)")
    max_context_length: int = Field(description="최대 컨텍스트 길이")
    supports_streaming: bool = Field(description="스트리밍 지원 여부")
    privacy_level: Literal["cloud", "local"] = Field(description="프라이버시 레벨")


class ServingSelection(BaseModel):
    """서빙 선택 결과를 표현하는 모델"""
    selected_endpoint: ServingEndpoint = Field(description="선택된 엔드포인트")
    selection_reason: str = Field(description="선택 이유")
    confidence: float = Field(ge=0.0, le=1.0, description="선택 신뢰도")
    fallback_endpoint: Optional[ServingEndpoint] = Field(
        default=None,
        description="대체 엔드포인트"
    )
    estimated_cost: float = Field(ge=0, description="예상 비용 (USD)")


class ServingResponse(BaseModel):
    """LLM 서빙 응답을 표현하는 모델"""
    content: str = Field(description="응답 내용")
    serving_type: ServingType = Field(description="사용된 서빙 타입")
    model_name: str = Field(description="사용된 모델 이름")
    latency_ms: int = Field(description="실제 지연시간 (밀리초)")
    tokens_used: int = Field(description="사용된 토큰 수")
    actual_cost: float = Field(description="실제 비용 (USD)")
    was_fallback: bool = Field(description="폴백 사용 여부")
    timestamp: datetime = Field(default_factory=datetime.now, description="응답 시간")


class ServiceHealth(BaseModel):
    """서비스 헬스 체크 결과를 표현하는 모델"""
    serving_type: ServingType = Field(description="서빙 타입")
    is_healthy: bool = Field(description="서비스 정상 여부")
    response_time_ms: Optional[int] = Field(default=None, description="응답 시간")
    error_message: Optional[str] = Field(default=None, description="오류 메시지")
    last_check: datetime = Field(default_factory=datetime.now, description="마지막 체크 시간")


class ConversationMessage(BaseModel):
    """대화 메시지를 표현하는 모델"""
    role: Literal["user", "assistant", "system"] = Field(description="메시지 역할")
    content: str = Field(description="메시지 내용")
    timestamp: datetime = Field(default_factory=datetime.now, description="메시지 생성 시간")
    serving_metadata: Optional[Dict] = Field(default=None, description="서빙 메타데이터")


class AgentResponse(BaseModel):
    """에이전트의 최종 응답을 표현하는 모델"""
    answer: str = Field(description="사용자에게 제공되는 답변")
    serving_info: ServingResponse = Field(description="서빙 정보")
    request_analysis: RequestCharacteristics = Field(description="요청 분석 결과")
    cost_savings: float = Field(description="로컬 사용으로 절감된 비용 (USD)")
    recommendation: Optional[str] = Field(
        default=None,
        description="사용자를 위한 추천 사항"
    )

## 3. LLM 서빙 엔드포인트 설정

In [4]:
# 서빙 엔드포인트 레지스트리
SERVING_ENDPOINTS = {
    "cloud_gpt4o_mini": ServingEndpoint(
        serving_type=ServingType.CLOUD,
        endpoint_name="OpenAI GPT-4o-mini",
        model_name="gpt-4o-mini",
        is_available=True,
        avg_latency_ms=800,
        cost_per_1k_tokens=0.00030,
        max_context_length=128000,
        supports_streaming=True,
        privacy_level="cloud"
    ),
    "local_ollama_llama3": ServingEndpoint(
        serving_type=ServingType.LOCAL,
        endpoint_name="Ollama exaone3.5",
        model_name="exaone3.5:7.8",
        is_available=True,  # 실제로는 Ollama 설치 여부에 따라 결정
        avg_latency_ms=300,
        cost_per_1k_tokens=0.0,  # 로컬은 무료
        max_context_length=8192,
        supports_streaming=True,
        privacy_level="local"
    )
}

# Ollama 로컬 서버 URL (기본값)
OLLAMA_BASE_URL = "http://localhost:11434"

print(f"총 {len(SERVING_ENDPOINTS)}개의 서빙 엔드포인트가 등록되었다.")
for endpoint_id, endpoint in SERVING_ENDPOINTS.items():
    print(f"  - {endpoint.endpoint_name} ({endpoint.serving_type.value}): "
          f"지연시간 {endpoint.avg_latency_ms}ms, "
          f"비용 ${endpoint.cost_per_1k_tokens:.5f}/1K tokens")

총 2개의 서빙 엔드포인트가 등록되었다.
  - OpenAI GPT-4o-mini (cloud): 지연시간 800ms, 비용 $0.00030/1K tokens
  - Ollama exaone3.5 (local): 지연시간 300ms, 비용 $0.00000/1K tokens


## 4. Memory 시스템 구현

In [5]:
class ServingMemory:
    """서빙 이력과 성능 통계를 관리하는 메모리 클래스"""
    
    def __init__(self, max_history: int = 20):
        self.messages: List[ConversationMessage] = []
        self.max_history = max_history
        self.serving_history: List[ServingResponse] = []
        self.service_stats: Dict[str, Dict] = {}
        
        # 각 서비스의 통계 초기화
        for endpoint_id in SERVING_ENDPOINTS.keys():
            self.service_stats[endpoint_id] = {
                "total_requests": 0,
                "successful_requests": 0,
                "failed_requests": 0,
                "total_latency_ms": 0,
                "total_tokens": 0,
                "total_cost": 0.0,
                "avg_latency_ms": 0,
                "success_rate": 0.0
            }
    
    def add_message(self, role: str, content: str, serving_metadata: Dict = None):
        """새로운 메시지를 메모리에 추가한다"""
        message = ConversationMessage(
            role=role,
            content=content,
            serving_metadata=serving_metadata
        )
        self.messages.append(message)
        
        # 최대 이력 수를 초과하면 오래된 메시지부터 제거한다
        if len(self.messages) > self.max_history:
            self.messages = self.messages[-self.max_history:]
    
    def record_serving_response(self, response: ServingResponse, success: bool):
        """
        서빙 응답을 기록하고 통계를 업데이트한다
        
        Args:
            response: 서빙 응답
            success: 성공 여부
        """
        self.serving_history.append(response)
        
        # 엔드포인트 ID 찾기
        endpoint_id = None
        for eid, endpoint in SERVING_ENDPOINTS.items():
            if (endpoint.serving_type == response.serving_type and 
                endpoint.model_name == response.model_name):
                endpoint_id = eid
                break
        
        if endpoint_id and endpoint_id in self.service_stats:
            stats = self.service_stats[endpoint_id]
            stats["total_requests"] += 1
            
            if success:
                stats["successful_requests"] += 1
            else:
                stats["failed_requests"] += 1
            
            stats["total_latency_ms"] += response.latency_ms
            stats["total_tokens"] += response.tokens_used
            stats["total_cost"] += response.actual_cost
            
            # 평균 계산
            if stats["total_requests"] > 0:
                stats["avg_latency_ms"] = stats["total_latency_ms"] / stats["total_requests"]
                stats["success_rate"] = stats["successful_requests"] / stats["total_requests"]
    
    def get_service_performance(self, endpoint_id: str) -> Dict:
        """특정 서비스의 성능 통계를 반환한다"""
        return self.service_stats.get(endpoint_id, {})
    
    def get_total_cost(self) -> float:
        """전체 서비스 사용 비용을 계산한다"""
        return sum(stats["total_cost"] for stats in self.service_stats.values())
    
    def get_cost_savings(self) -> float:
        """
        로컬 서비스 사용으로 절감된 비용을 계산한다
        
        Returns:
            절감된 비용 (USD)
        """
        local_requests = 0
        local_tokens = 0
        
        # 로컬 서비스 사용량 집계
        for endpoint_id, stats in self.service_stats.items():
            endpoint = SERVING_ENDPOINTS[endpoint_id]
            if endpoint.serving_type == ServingType.LOCAL:
                local_requests += stats["total_requests"]
                local_tokens += stats["total_tokens"]
        
        # 클라우드 비용으로 환산
        cloud_endpoint = SERVING_ENDPOINTS["cloud_gpt4o_mini"]
        would_be_cost = (local_tokens / 1000) * cloud_endpoint.cost_per_1k_tokens
        
        return would_be_cost
    
    def get_context(self) -> List[Dict]:
        """OpenAI API 형식으로 대화 이력을 반환한다"""
        context = []
        for msg in self.messages:
            context.append({
                "role": msg.role,
                "content": msg.content
            })
        return context
    
    def get_summary(self) -> str:
        """메모리 상태 요약을 반환한다"""
        total_requests = sum(stats["total_requests"] for stats in self.service_stats.values())
        total_cost = self.get_total_cost()
        cost_savings = self.get_cost_savings()
        return f"총 요청: {total_requests}회, 비용: ${total_cost:.4f}, 절감: ${cost_savings:.4f}"
    
    def clear(self):
        """모든 이력을 삭제한다"""
        self.messages = []
        self.serving_history = []

# 메모리 인스턴스 생성
memory = ServingMemory(max_history=20)

# 시스템 메시지 추가
memory.add_message(
    "system",
    "당신은 커피 키오스크의 주문 도우미다. 고객의 질문에 친절하고 정확하게 답변하라."
)

print("서빙 메모리가 초기화되었다.")

서빙 메모리가 초기화되었다.


## 5. 요청 특성 분석 시스템 구현

In [6]:
class RequestAnalyzer:
    """요청 특성 분석 클래스"""
    
    def __init__(self):
        self.analyzer_model = "gpt-4o-mini"
    
    def analyze_request(self, query: str, context: List[Dict] = None) -> RequestCharacteristics:
        """
        사용자 요청의 특성을 분석한다
        
        Args:
            query: 사용자 질문
            context: 대화 컨텍스트
        
        Returns:
            RequestCharacteristics 객체
        """
        analysis_prompt = f"""
다음 커피 키오스크 고객 질문의 특성을 분석하라.

질문: "{query}"

다음 기준으로 평가하라:

1. complexity (복잡도):
   - simple: 단순 질문 (가격, 메뉴 확인, 인사)
   - moderate: 중간 복잡도 (메뉴 비교, 추천)
   - complex: 복잡한 질문 (맞춤형 조합, 다단계 추론)

2. privacy_sensitivity (프라이버시 민감도):
   - low: 민감 정보 없음
   - medium: 선호도 정보
   - high: 개인정보, 결제정보, 건강정보

3. response_time_requirement (응답 시간 요구사항):
   - fast: 즉각 응답 필요 (단순 확인)
   - balanced: 균형 (일반 질문)
   - quality: 품질 우선 (복잡한 추천)

4. contains_sensitive_data: 민감 정보 포함 여부 (true/false)

5. estimated_tokens: 예상 토큰 수 (응답 길이 예측)

6. reasoning: 분석 근거 (한 문장)

JSON 형식으로만 답변하라:
{{
  "complexity": "simple|moderate|complex",
  "privacy_sensitivity": "low|medium|high",
  "response_time_requirement": "fast|balanced|quality",
  "contains_sensitive_data": true|false,
  "estimated_tokens": 숫자,
  "reasoning": "분석 근거"
}}
"""
        
        messages = context or []
        messages.append({"role": "user", "content": analysis_prompt})
        
        response = client.chat.completions.create(
            model=self.analyzer_model,
            messages=messages,
            temperature=0.1,
            response_format={"type": "json_object"}
        )
        
        result = json.loads(response.choices[0].message.content)
        
        return RequestCharacteristics(**result)

# 요청 분석기 인스턴스 생성
request_analyzer = RequestAnalyzer()
print("요청 분석 시스템이 초기화되었다.")

요청 분석 시스템이 초기화되었다.


## 6. Function Calling & Tool 정의

In [7]:
# 서빙 선택을 위한 도구 정의
SERVING_SELECTION_TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "select_cloud_serving",
            "description": "클라우드 LLM 서빙을 선택한다. 복잡한 질문, 높은 품질 요구, 대용량 컨텍스트가 필요한 경우 사용한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "reason": {
                        "type": "string",
                        "description": "클라우드 서빙을 선택한 이유"
                    },
                    "confidence": {
                        "type": "number",
                        "description": "선택 신뢰도 (0.0-1.0)"
                    }
                },
                "required": ["reason", "confidence"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "select_local_serving",
            "description": "로컬 LLM 서빙을 선택한다. 단순한 질문, 빠른 응답, 프라이버시 보호, 비용 절감이 중요한 경우 사용한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "reason": {
                        "type": "string",
                        "description": "로컬 서빙을 선택한 이유"
                    },
                    "confidence": {
                        "type": "number",
                        "description": "선택 신뢰도 (0.0-1.0)"
                    }
                },
                "required": ["reason", "confidence"]
            }
        }
    }
]

# 도구 이름과 엔드포인트 ID 매핑
TOOL_TO_ENDPOINT_MAP = {
    "select_cloud_serving": "cloud_gpt4o_mini",
    "select_local_serving": "local_ollama_llama3"
}

print(f"총 {len(SERVING_SELECTION_TOOLS)}개의 서빙 선택 도구가 정의되었다.")

총 2개의 서빙 선택 도구가 정의되었다.


## 7. 서빙 라우터 구현

In [8]:
class ServingRouter:
    """LLM 서빙 라우터 클래스"""
    
    def __init__(self, memory: ServingMemory):
        self.memory = memory
        self.router_model = "gpt-4o-mini"
    
    def route(self, query: str, characteristics: RequestCharacteristics) -> ServingSelection:
        """
        요청 특성을 바탕으로 최적의 서빙을 선택한다
        
        Args:
            query: 사용자 질문
            characteristics: 요청 특성 분석 결과
        
        Returns:
            ServingSelection 객체
        """
        # 서비스별 과거 성능 정보 수집
        cloud_perf = self.memory.get_service_performance("cloud_gpt4o_mini")
        local_perf = self.memory.get_service_performance("local_ollama_llama3")
        
        routing_prompt = f"""
커피 키오스크 질문에 대해 최적의 LLM 서빙을 선택하라.

질문: "{query}"

요청 특성:
- 복잡도: {characteristics.complexity}
- 프라이버시 민감도: {characteristics.privacy_sensitivity}
- 응답 시간 요구: {characteristics.response_time_requirement}
- 민감 정보 포함: {characteristics.contains_sensitive_data}
- 예상 토큰: {characteristics.estimated_tokens}
- 분석 근거: {characteristics.reasoning}

사용 가능한 서빙:

1. Cloud Serving (OpenAI GPT-4o-mini):
   - 비용: $0.00030/1K tokens
   - 평균 지연: 800ms
   - 과거 성공률: {cloud_perf.get('success_rate', 0):.1%}
   - 과거 평균 지연: {cloud_perf.get('avg_latency_ms', 0):.0f}ms
   - 장점: 높은 품질, 복잡한 추론, 대용량 컨텍스트
   - 단점: 비용 발생, 데이터 외부 전송

2. Local Serving (Ollama Llama3):
   - 비용: $0 (무료)
   - 평균 지연: 300ms
   - 과거 성공률: {local_perf.get('success_rate', 0):.1%}
   - 과거 평균 지연: {local_perf.get('avg_latency_ms', 0):.0f}ms
   - 장점: 무료, 빠른 응답, 프라이버시 보호
   - 단점: 제한된 성능

선택 기준:
1. 프라이버시 민감도가 high이면 Local 우선
2. 복잡도가 complex이면 Cloud 우선
3. 응답 시간이 fast이면 Local 우선
4. 비용 효율을 고려

가장 적합한 서빙을 선택하는 함수를 호출하라.
"""
        
        # Function Calling으로 서빙 선택
        response = client.chat.completions.create(
            model=self.router_model,
            messages=[{"role": "user", "content": routing_prompt}],
            tools=SERVING_SELECTION_TOOLS,
            tool_choice="required"
        )
        
        # 도구 호출 결과 파싱
        tool_call = response.choices[0].message.tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        # 선택된 엔드포인트 가져오기
        selected_endpoint_id = TOOL_TO_ENDPOINT_MAP[function_name]
        selected_endpoint = SERVING_ENDPOINTS[selected_endpoint_id]
        
        # 대체 엔드포인트 설정
        fallback_endpoint_id = None
        if selected_endpoint.serving_type == ServingType.CLOUD:
            fallback_endpoint_id = "local_ollama_llama3"
        else:
            fallback_endpoint_id = "cloud_gpt4o_mini"
        fallback_endpoint = SERVING_ENDPOINTS[fallback_endpoint_id]
        
        # 예상 비용 계산
        estimated_cost = (characteristics.estimated_tokens / 1000) * selected_endpoint.cost_per_1k_tokens
        
        return ServingSelection(
            selected_endpoint=selected_endpoint,
            selection_reason=function_args["reason"],
            confidence=function_args["confidence"],
            fallback_endpoint=fallback_endpoint,
            estimated_cost=estimated_cost
        )

# 서빙 라우터 인스턴스 생성
serving_router = ServingRouter(memory)
print("서빙 라우터가 초기화되었다.")

서빙 라우터가 초기화되었다.


## 8. LLM 서빙 클라이언트 구현

In [9]:
class LLMServingClient:
    """LLM 서빙 클라이언트 클래스"""
    
    def __init__(self):
        self.ollama_available = self._check_ollama_availability()
    
    def _check_ollama_availability(self) -> bool:
        """Ollama 로컬 서버가 사용 가능한지 확인한다"""
        try:
            response = requests.get(f"{OLLAMA_BASE_URL}/api/tags", timeout=2)
            return response.status_code == 200
        except:
            return False
    
    def generate_cloud(self, query: str, context: List[Dict], endpoint: ServingEndpoint) -> tuple[str, int, int]:
        """
        클라우드 LLM으로 응답을 생성한다
        
        Args:
            query: 사용자 질문
            context: 대화 컨텍스트
            endpoint: 서빙 엔드포인트
        
        Returns:
            (응답 내용, 지연시간 ms, 토큰 수)
        """
        messages = context.copy()
        messages.append({"role": "user", "content": query})
        
        start_time = time.time()
        
        response = client.chat.completions.create(
            model=endpoint.model_name,
            messages=messages,
            temperature=0.7
        )
        
        end_time = time.time()
        latency_ms = int((end_time - start_time) * 1000)
        
        content = response.choices[0].message.content.strip()
        tokens_used = response.usage.total_tokens
        
        return content, latency_ms, tokens_used
    
    def generate_local(self, query: str, context: List[Dict], endpoint: ServingEndpoint) -> tuple[str, int, int]:
        """
        로컬 LLM으로 응답을 생성한다
        
        Args:
            query: 사용자 질문
            context: 대화 컨텍스트
            endpoint: 서빙 엔드포인트
        
        Returns:
            (응답 내용, 지연시간 ms, 토큰 수)
        """
        # Ollama가 사용 불가능하면 예외 발생
        if not self.ollama_available:
            raise Exception("Ollama 로컬 서버를 사용할 수 없다")
        
        # 컨텍스트를 Ollama 형식으로 변환
        prompt = self._build_ollama_prompt(query, context)
        
        start_time = time.time()
        
        # Ollama API 호출
        response = requests.post(
            f"{OLLAMA_BASE_URL}/api/generate",
            json={
                "model": endpoint.model_name,
                "prompt": prompt,
                "stream": False
            },
            timeout=30
        )
        
        end_time = time.time()
        latency_ms = int((end_time - start_time) * 1000)
        
        result = response.json()
        content = result["response"].strip()
        
        # Ollama는 토큰 수를 반환하지 않으므로 대략 추정
        tokens_used = len(content.split()) * 2  # 간단한 추정
        
        return content, latency_ms, tokens_used
    
    def _build_ollama_prompt(self, query: str, context: List[Dict]) -> str:
        """Ollama용 프롬프트를 구성한다"""
        prompt_parts = []
        
        for msg in context:
            role = msg["role"]
            content = msg["content"]
            
            if role == "system":
                prompt_parts.append(f"시스템: {content}")
            elif role == "user":
                prompt_parts.append(f"사용자: {content}")
            elif role == "assistant":
                prompt_parts.append(f"도우미: {content}")
        
        prompt_parts.append(f"사용자: {query}")
        prompt_parts.append("도우미:")
        
        return "\n\n".join(prompt_parts)
    
    def generate_mock_local(self, query: str, context: List[Dict], endpoint: ServingEndpoint) -> tuple[str, int, int]:
        """
        Ollama가 없을 때 사용하는 모의 로컬 응답 생성
        (실제로는 gpt-4o-mini를 사용하지만 로컬처럼 시뮬레이션)
        
        Returns:
            (응답 내용, 지연시간 ms, 토큰 수)
        """
        print("  [시뮬레이션] Ollama를 사용할 수 없어 모의 로컬 응답을 생성한다")
        
        messages = context.copy()
        messages.append({"role": "user", "content": query})
        
        start_time = time.time()
        
        # 간단한 응답 생성 (로컬 모델은 더 간결함)
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.5,
            max_tokens=150  # 로컬 모델은 응답이 더 짧음
        )
        
        # 로컬 모델의 더 빠른 응답 시간을 시뮬레이션
        time.sleep(0.1)  # 로컬의 빠른 응답 시뮬레이션
        
        end_time = time.time()
        latency_ms = int((end_time - start_time) * 1000)
        
        content = response.choices[0].message.content.strip()
        tokens_used = response.usage.total_tokens
        
        return content, latency_ms, tokens_used

# LLM 서빙 클라이언트 인스턴스 생성
llm_client = LLMServingClient()
print(f"LLM 서빙 클라이언트가 초기화되었다.")
print(f"  - Ollama 사용 가능: {'예' if llm_client.ollama_available else '아니오 (모의 로컬 모드 사용)'}")

LLM 서빙 클라이언트가 초기화되었다.
  - Ollama 사용 가능: 예


## 9. Validation 시스템 구현

In [10]:
class ServingValidationSystem:
    """서빙 검증 시스템 클래스"""
    
    def __init__(self, llm_client: LLMServingClient):
        self.llm_client = llm_client
        self.health_cache: Dict[str, ServiceHealth] = {}
    
    def check_service_health(self, endpoint: ServingEndpoint) -> ServiceHealth:
        """
        서비스의 헬스를 체크한다
        
        Args:
            endpoint: 체크할 엔드포인트
        
        Returns:
            ServiceHealth 객체
        """
        start_time = time.time()
        
        try:
            if endpoint.serving_type == ServingType.CLOUD:
                # 클라우드는 항상 사용 가능하다고 가정
                is_healthy = True
                error_message = None
            else:
                # 로컬 서비스 체크
                is_healthy = self.llm_client.ollama_available
                error_message = None if is_healthy else "Ollama 서버에 연결할 수 없다"
            
            end_time = time.time()
            response_time_ms = int((end_time - start_time) * 1000)
            
            health = ServiceHealth(
                serving_type=endpoint.serving_type,
                is_healthy=is_healthy,
                response_time_ms=response_time_ms,
                error_message=error_message
            )
            
            # 캐시에 저장
            cache_key = f"{endpoint.serving_type.value}_{endpoint.model_name}"
            self.health_cache[cache_key] = health
            
            return health
            
        except Exception as e:
            return ServiceHealth(
                serving_type=endpoint.serving_type,
                is_healthy=False,
                response_time_ms=None,
                error_message=str(e)
            )
    
    def validate_response(self, response_content: str, query: str) -> tuple[bool, str]:
        """
        생성된 응답의 유효성을 검증한다
        
        Args:
            response_content: 응답 내용
            query: 원본 질문
        
        Returns:
            (유효성, 검증 메시지)
        """
        # 기본 검증
        if not response_content or len(response_content.strip()) < 5:
            return False, "응답이 너무 짧거나 비어있다"
        
        # 에러 메시지 체크
        error_indicators = ["오류", "에러", "실패", "불가능"]
        if any(indicator in response_content[:50] for indicator in error_indicators):
            return False, "응답에 오류 메시지가 포함되어 있다"
        
        return True, "응답이 유효하다"

# 검증 시스템 인스턴스 생성
validation_system = ServingValidationSystem(llm_client)
print("서빙 검증 시스템이 초기화되었다.")

서빙 검증 시스템이 초기화되었다.


## 10. Recovery 메커니즘 구현

In [11]:
class ServingRecoverySystem:
    """서빙 복구 메커니즘 시스템 클래스"""
    
    def __init__(self, llm_client: LLMServingClient, validation_system: ServingValidationSystem):
        self.llm_client = llm_client
        self.validation_system = validation_system
        self.fallback_history = []
    
    def attempt_with_fallback(
        self,
        query: str,
        context: List[Dict],
        selection: ServingSelection
    ) -> tuple[ServingResponse, bool]:
        """
        주 서비스로 시도하고 실패 시 대체 서비스로 폴백한다
        
        Args:
            query: 사용자 질문
            context: 대화 컨텍스트
            selection: 서빙 선택 결과
        
        Returns:
            (ServingResponse, 폴백 사용 여부)
        """
        # 주 서비스 시도
        primary_endpoint = selection.selected_endpoint
        
        print(f"\n주 서비스 시도: {primary_endpoint.endpoint_name}")
        
        try:
            response = self._generate_response(query, context, primary_endpoint)
            
            # 응답 검증
            is_valid, validation_msg = self.validation_system.validate_response(
                response.content,
                query
            )
            
            if is_valid:
                print(f"  성공: {validation_msg}")
                return response, False
            else:
                print(f"  검증 실패: {validation_msg}")
                raise Exception(validation_msg)
        
        except Exception as e:
            print(f"  오류 발생: {str(e)}")
            
            # 대체 서비스로 폴백
            if selection.fallback_endpoint:
                print(f"\n대체 서비스로 폴백: {selection.fallback_endpoint.endpoint_name}")
                
                # 폴백 이력 기록
                self.fallback_history.append({
                    "timestamp": datetime.now(),
                    "from": primary_endpoint.endpoint_name,
                    "to": selection.fallback_endpoint.endpoint_name,
                    "reason": str(e)
                })
                
                try:
                    response = self._generate_response(query, context, selection.fallback_endpoint)
                    print(f"  대체 서비스 성공")
                    response.was_fallback = True
                    return response, True
                    
                except Exception as fallback_error:
                    print(f"  대체 서비스도 실패: {str(fallback_error)}")
                    raise Exception(f"모든 서비스가 실패했다: {str(fallback_error)}")
            else:
                raise Exception(f"대체 서비스가 없다: {str(e)}")
    
    def _generate_response(
        self,
        query: str,
        context: List[Dict],
        endpoint: ServingEndpoint
    ) -> ServingResponse:
        """
        지정된 엔드포인트로 응답을 생성한다
        
        Returns:
            ServingResponse 객체
        """
        # 서비스 헬스 체크
        health = self.validation_system.check_service_health(endpoint)
        if not health.is_healthy:
            raise Exception(f"서비스가 사용 불가능하다: {health.error_message}")
        
        # 응답 생성
        if endpoint.serving_type == ServingType.CLOUD:
            content, latency_ms, tokens_used = self.llm_client.generate_cloud(
                query, context, endpoint
            )
        else:
            # Ollama가 사용 가능하면 실제 로컬, 아니면 모의 로컬
            if self.llm_client.ollama_available:
                content, latency_ms, tokens_used = self.llm_client.generate_local(
                    query, context, endpoint
                )
            else:
                content, latency_ms, tokens_used = self.llm_client.generate_mock_local(
                    query, context, endpoint
                )
        
        # 비용 계산
        actual_cost = (tokens_used / 1000) * endpoint.cost_per_1k_tokens
        
        return ServingResponse(
            content=content,
            serving_type=endpoint.serving_type,
            model_name=endpoint.model_name,
            latency_ms=latency_ms,
            tokens_used=tokens_used,
            actual_cost=actual_cost,
            was_fallback=False
        )
    
    def get_fallback_stats(self) -> Dict:
        """폴백 통계를 반환한다"""
        if not self.fallback_history:
            return {"count": 0, "most_common_reason": None}
        
        reasons = [f["reason"] for f in self.fallback_history]
        most_common = max(set(reasons), key=reasons.count) if reasons else None
        
        return {
            "count": len(self.fallback_history),
            "most_common_reason": most_common
        }

# 복구 시스템 인스턴스 생성
recovery_system = ServingRecoverySystem(llm_client, validation_system)
print("서빙 복구 시스템이 초기화되었다.")

서빙 복구 시스템이 초기화되었다.


## 11. Feedback 시스템 구현

In [12]:
class ServingFeedbackSystem:
    """서빙 피드백 시스템 클래스"""
    
    def __init__(self, memory: ServingMemory):
        self.memory = memory
    
    def analyze_service_efficiency(self) -> Dict[str, Dict]:
        """
        각 서비스의 효율성을 분석한다
        
        Returns:
            서비스별 효율성 분석 결과
        """
        efficiency = {}
        
        for endpoint_id, endpoint in SERVING_ENDPOINTS.items():
            stats = self.memory.get_service_performance(endpoint_id)
            
            if stats["total_requests"] == 0:
                efficiency[endpoint_id] = {
                    "score": 0.0,
                    "cost_efficiency": 0.0,
                    "speed_score": 0.0,
                    "reliability_score": 0.0
                }
                continue
            
            # 비용 효율성 (낮을수록 좋음, 로컬은 만점)
            if endpoint.cost_per_1k_tokens == 0:
                cost_efficiency = 1.0
            else:
                cost_efficiency = max(0, 1.0 - (stats["total_cost"] / 1.0))  # 1달러 기준
            
            # 속도 점수 (빠를수록 좋음)
            target_latency = 500  # 목표 지연시간 (ms)
            speed_score = max(0, 1.0 - (stats["avg_latency_ms"] / target_latency))
            
            # 신뢰도 점수
            reliability_score = stats["success_rate"]
            
            # 종합 효율성 점수
            overall_score = (
                cost_efficiency * 0.3 +
                speed_score * 0.3 +
                reliability_score * 0.4
            )
            
            efficiency[endpoint_id] = {
                "score": overall_score,
                "cost_efficiency": cost_efficiency,
                "speed_score": speed_score,
                "reliability_score": reliability_score
            }
        
        return efficiency
    
    def generate_optimization_recommendations(self) -> List[str]:
        """
        성능 분석을 바탕으로 최적화 제안을 생성한다
        
        Returns:
            최적화 제안 리스트
        """
        recommendations = []
        
        # 서비스 효율성 분석
        efficiency = self.analyze_service_efficiency()
        
        # 비용 분석
        total_cost = self.memory.get_total_cost()
        cost_savings = self.memory.get_cost_savings()
        
        if total_cost > 1.0:  # 1달러 이상 사용
            recommendations.append(
                f"총 비용이 ${total_cost:.4f}다. "
                f"로컬 서비스를 더 많이 사용하여 비용을 절감하라."
            )
        
        if cost_savings > 0.5:  # 50센트 이상 절감
            recommendations.append(
                f"로컬 서비스 사용으로 ${cost_savings:.4f}를 절감했다. "
                f"현재 전략을 유지하라."
            )
        
        # 서비스별 성능 분석
        for endpoint_id, eff in efficiency.items():
            endpoint = SERVING_ENDPOINTS[endpoint_id]
            stats = self.memory.get_service_performance(endpoint_id)
            
            if stats["total_requests"] == 0:
                continue
            
            # 성공률이 낮은 경우
            if eff["reliability_score"] < 0.8:
                recommendations.append(
                    f"{endpoint.endpoint_name}의 성공률이 {eff['reliability_score']:.1%}로 낮다. "
                    f"서비스 상태를 점검하라."
                )
            
            # 속도가 느린 경우
            if eff["speed_score"] < 0.5:
                recommendations.append(
                    f"{endpoint.endpoint_name}의 응답 속도가 느리다 (평균 {stats['avg_latency_ms']:.0f}ms). "
                    f"더 빠른 서비스를 우선 사용하라."
                )
        
        # 로컬 서비스 활용도 분석
        local_stats = self.memory.get_service_performance("local_ollama_llama3")
        cloud_stats = self.memory.get_service_performance("cloud_gpt4o_mini")
        
        total_requests = local_stats["total_requests"] + cloud_stats["total_requests"]
        
        if total_requests > 0:
            local_ratio = local_stats["total_requests"] / total_requests
            
            if local_ratio < 0.3:
                recommendations.append(
                    f"로컬 서비스 사용률이 {local_ratio:.1%}로 낮다. "
                    f"간단한 질문에 로컬 서비스를 더 활용하여 비용을 절감하라."
                )
            elif local_ratio > 0.7:
                recommendations.append(
                    f"로컬 서비스 사용률이 {local_ratio:.1%}로 높다. "
                    f"복잡한 질문에는 클라우드 서비스를 사용하여 품질을 향상하라."
                )
        
        return recommendations if recommendations else ["현재 서빙 전략이 최적이다."]
    
    def get_performance_summary(self) -> Dict:
        """성능 요약을 반환한다"""
        cloud_stats = self.memory.get_service_performance("cloud_gpt4o_mini")
        local_stats = self.memory.get_service_performance("local_ollama_llama3")
        
        return {
            "cloud": {
                "requests": cloud_stats["total_requests"],
                "success_rate": cloud_stats["success_rate"],
                "avg_latency_ms": cloud_stats["avg_latency_ms"],
                "total_cost": cloud_stats["total_cost"]
            },
            "local": {
                "requests": local_stats["total_requests"],
                "success_rate": local_stats["success_rate"],
                "avg_latency_ms": local_stats["avg_latency_ms"],
                "total_cost": local_stats["total_cost"]
            },
            "total_cost": self.memory.get_total_cost(),
            "cost_savings": self.memory.get_cost_savings()
        }

# 피드백 시스템 인스턴스 생성
feedback_system = ServingFeedbackSystem(memory)
print("서빙 피드백 시스템이 초기화되었다.")

서빙 피드백 시스템이 초기화되었다.


## 12. Router LLM Serving Agent 통합 구현

In [13]:
class RouterLLMServingAgent:
    """Router LLM Serving Agent 메인 클래스"""
    
    def __init__(
        self,
        request_analyzer: RequestAnalyzer,
        serving_router: ServingRouter,
        recovery_system: ServingRecoverySystem,
        memory: ServingMemory,
        feedback: ServingFeedbackSystem
    ):
        self.request_analyzer = request_analyzer
        self.serving_router = serving_router
        self.recovery_system = recovery_system
        self.memory = memory
        self.feedback = feedback
    
    def process_query(self, user_query: str) -> AgentResponse:
        """
        사용자 질문을 처리하고 최종 응답을 생성한다
        
        Args:
            user_query: 사용자 질문
        
        Returns:
            AgentResponse 객체
        """
        print(f"\n{'='*60}")
        print(f"사용자 질문: {user_query}")
        print(f"{'='*60}")
        
        # 1. 사용자 메시지를 메모리에 추가
        self.memory.add_message("user", user_query)
        
        # 2. 요청 특성 분석
        print("\n단계 1: 요청 특성 분석")
        context = self.memory.get_context()
        characteristics = self.request_analyzer.analyze_request(user_query, context)
        print(f"  - 복잡도: {characteristics.complexity}")
        print(f"  - 프라이버시: {characteristics.privacy_sensitivity}")
        print(f"  - 응답 시간: {characteristics.response_time_requirement}")
        print(f"  - 민감 정보: {'포함' if characteristics.contains_sensitive_data else '없음'}")
        print(f"  - 분석 근거: {characteristics.reasoning}")
        
        # 3. 서빙 선택
        print("\n단계 2: 서빙 선택")
        selection = self.serving_router.route(user_query, characteristics)
        print(f"  - 선택된 서빙: {selection.selected_endpoint.endpoint_name}")
        print(f"  - 선택 이유: {selection.selection_reason}")
        print(f"  - 신뢰도: {selection.confidence:.2f}")
        print(f"  - 예상 비용: ${selection.estimated_cost:.6f}")
        
        # 4. 응답 생성 (폴백 포함)
        print("\n단계 3: 응답 생성")
        serving_response, was_fallback = self.recovery_system.attempt_with_fallback(
            user_query,
            context,
            selection
        )
        
        print(f"\n응답 생성 완료:")
        print(f"  - 사용된 서빙: {serving_response.serving_type.value}")
        print(f"  - 모델: {serving_response.model_name}")
        print(f"  - 지연시간: {serving_response.latency_ms}ms")
        print(f"  - 토큰: {serving_response.tokens_used}개")
        print(f"  - 비용: ${serving_response.actual_cost:.6f}")
        print(f"  - 폴백 사용: {'예' if was_fallback else '아니오'}")
        
        # 5. 서빙 응답 기록
        self.memory.record_serving_response(serving_response, success=True)
        
        # 6. 비용 절감 계산
        cost_savings = self.memory.get_cost_savings()
        
        # 7. 메모리에 답변 추가
        self.memory.add_message(
            "assistant",
            serving_response.content,
            serving_metadata={
                "serving_type": serving_response.serving_type.value,
                "latency_ms": serving_response.latency_ms,
                "cost": serving_response.actual_cost
            }
        )
        
        # 8. 사용자 추천 생성
        recommendation = None
        if serving_response.serving_type == ServingType.LOCAL:
            recommendation = "로컬 서비스를 사용하여 비용을 절감했다."
        elif characteristics.privacy_sensitivity == "high":
            recommendation = "민감한 정보는 로컬 서비스 사용을 권장한다."
        
        # 9. 최종 응답 생성
        return AgentResponse(
            answer=serving_response.content,
            serving_info=serving_response,
            request_analysis=characteristics,
            cost_savings=cost_savings,
            recommendation=recommendation
        )

# Router LLM Serving Agent 인스턴스 생성
agent = RouterLLMServingAgent(
    request_analyzer=request_analyzer,
    serving_router=serving_router,
    recovery_system=recovery_system,
    memory=memory,
    feedback=feedback_system
)

print("\nRouter LLM Serving Agent가 초기화되었다.")


Router LLM Serving Agent가 초기화되었다.


## 13. 에이전트 테스트

In [14]:
# 테스트 질문 리스트 (다양한 특성)
test_queries = [
    "안녕하세요",  # Simple, Local 예상
    "아메리카노 가격은 얼마인가요?",  # Simple, Local 예상
    "저는 카페인에 민감한데, 맛있는 음료를 추천해주세요",  # Moderate, 건강정보 포함
    "회의 전에 집중력을 높이면서도 오후까지 지속 가능한 에너지를 줄 수 있는 음료 조합을 추천해주세요. 단, 설탕은 적게 넣고 싶어요.",  # Complex, Cloud 예상
]

# 각 질문에 대해 에이전트 실행
for i, query in enumerate(test_queries, 1):
    print(f"\n\n{'#'*60}")
    print(f"테스트 {i}/{len(test_queries)}")
    print(f"{'#'*60}")
    
    response = agent.process_query(query)
    
    print(f"\n{'='*60}")
    print(f"최종 응답")
    print(f"{'='*60}")
    print(f"답변: {response.answer}")
    print(f"\n서빙 정보:")
    print(f"  - 타입: {response.serving_info.serving_type.value}")
    print(f"  - 모델: {response.serving_info.model_name}")
    print(f"  - 지연시간: {response.serving_info.latency_ms}ms")
    print(f"  - 비용: ${response.serving_info.actual_cost:.6f}")
    print(f"\n누적 비용 절감: ${response.cost_savings:.6f}")
    if response.recommendation:
        print(f"추천 사항: {response.recommendation}")



############################################################
테스트 1/4
############################################################

사용자 질문: 안녕하세요

단계 1: 요청 특성 분석
  - 복잡도: simple
  - 프라이버시: low
  - 응답 시간: fast
  - 민감 정보: 없음
  - 분석 근거: 고객의 인사는 단순한 질문으로, 민감한 정보가 없고 즉각적인 응답이 필요하다.

단계 2: 서빙 선택
  - 선택된 서빙: Ollama exaone3.5
  - 선택 이유: 요청이 단순하고 응답 시간이 빠르며 비용이 발생하지 않기 때문에 로컬 서빙을 선택함.
  - 신뢰도: 0.90
  - 예상 비용: $0.000000

단계 3: 응답 생성

주 서비스 시도: Ollama exaone3.5
  오류 발생: 'response'

대체 서비스로 폴백: OpenAI GPT-4o-mini
  대체 서비스 성공

응답 생성 완료:
  - 사용된 서빙: cloud
  - 모델: gpt-4o-mini
  - 지연시간: 4174ms
  - 토큰: 449개
  - 비용: $0.000135
  - 폴백 사용: 예

최종 응답
답변: {
  "complexity": "simple",
  "privacy_sensitivity": "low",
  "response_time_requirement": "fast",
  "contains_sensitive_data": false,
  "estimated_tokens": 2,
  "reasoning": "단순한 인사말로, 복잡한 정보나 프라이버시 민감도가 없기 때문에 간단한 질문으로 분류됨."
}

서빙 정보:
  - 타입: cloud
  - 모델: gpt-4o-mini
  - 지연시간: 4174ms
  - 비용: $0.000135

누적 비용 절감: $0.000000


##############################

## 14. 대화형 인터페이스

In [16]:
def chat_interface():
    """대화형 인터페이스 함수"""
    print("\n" + "="*60)
    print("커피 키오스크 Router LLM Serving Agent")
    print("종료하려면 'quit' 또는 'exit'를 입력하세요.")
    print("="*60 + "\n")
    
    while True:
        user_input = input("\n질문: ").strip()
        
        if user_input.lower() in ['quit', 'exit', '종료']:
            print("\n대화를 종료한다.")
            # 최종 통계 출력
            summary = feedback_system.get_performance_summary()
            print(f"\n총 비용: ${summary['total_cost']:.6f}")
            print(f"비용 절감: ${summary['cost_savings']:.6f}")
            break
        
        if not user_input:
            continue
        
        response = agent.process_query(user_input)
        print(f"\n에이전트: {response.answer}")
        print(f"\n[{response.serving_info.serving_type.value} | "
              f"{response.serving_info.latency_ms}ms | "
              f"${response.serving_info.actual_cost:.6f}]")

# 대화 시작 (주석을 제거하여 실행)
chat_interface()


커피 키오스크 Router LLM Serving Agent
종료하려면 'quit' 또는 'exit'를 입력하세요.




질문:  라떼 3잔을 주문하고 싶어



사용자 질문: 라떼 3잔을 주문하고 싶어

단계 1: 요청 특성 분석
  - 복잡도: simple
  - 프라이버시: low
  - 응답 시간: fast
  - 민감 정보: 없음
  - 분석 근거: 단순한 주문 요청으로 복잡하지 않으며, 민감한 정보가 포함되어 있지 않기 때문에 즉각적인 응답이 필요하다.

단계 2: 서빙 선택
  - 선택된 서빙: Ollama exaone3.5
  - 선택 이유: 단순한 주문 요청이므로 빠른 응답이 필요하고, 비용이 발생하지 않으며 프라이버시를 보호할 수 있다.
  - 신뢰도: 0.90
  - 예상 비용: $0.000000

단계 3: 응답 생성

주 서비스 시도: Ollama exaone3.5
  오류 발생: 'response'

대체 서비스로 폴백: OpenAI GPT-4o-mini
  대체 서비스 성공

응답 생성 완료:
  - 사용된 서빙: cloud
  - 모델: gpt-4o-mini
  - 지연시간: 1834ms
  - 토큰: 908개
  - 비용: $0.000272
  - 폴백 사용: 예

에이전트: {
  "complexity": "simple",
  "privacy_sensitivity": "low",
  "response_time_requirement": "fast",
  "contains_sensitive_data": false,
  "estimated_tokens": 10,
  "reasoning": "단순한 주문 요청으로, 복잡한 정보나 민감한 데이터가 포함되어 있지 않기 때문입니다."
}

[cloud | 1834ms | $0.000272]



질문:  나의 이름은 고우주이야



사용자 질문: 나의 이름은 고우주이야

단계 1: 요청 특성 분석
  - 복잡도: simple
  - 프라이버시: low
  - 응답 시간: fast
  - 민감 정보: 없음
  - 분석 근거: 단순히 자신의 이름을 언급하는 내용으로, 복잡한 질문이나 민감한 정보가 포함되어 있지 않기 때문입니다.

단계 2: 서빙 선택
  - 선택된 서빙: Ollama exaone3.5
  - 선택 이유: 단순한 질문으로 빠른 응답과 비용 절감이 중요하기 때문에 로컬 서빙을 선택함.
  - 신뢰도: 0.90
  - 예상 비용: $0.000000

단계 3: 응답 생성

주 서비스 시도: Ollama exaone3.5
  오류 발생: 'response'

대체 서비스로 폴백: OpenAI GPT-4o-mini
  대체 서비스 성공

응답 생성 완료:
  - 사용된 서빙: cloud
  - 모델: gpt-4o-mini
  - 지연시간: 1620ms
  - 토큰: 1000개
  - 비용: $0.000300
  - 폴백 사용: 예

에이전트: {
  "complexity": "simple",
  "privacy_sensitivity": "low",
  "response_time_requirement": "fast",
  "contains_sensitive_data": false,
  "estimated_tokens": 8,
  "reasoning": "단순히 자신의 이름을 소개하는 내용으로, 복잡한 질문이 아니고 민감한 정보도 포함되어 있지 않습니다."
}

[cloud | 1620ms | $0.000300]



질문:  라떼에 샷2잔, 우유 대신 두유, 라지 사이즈로 3잔 추가해줘



사용자 질문: 라떼에 샷2잔, 우유 대신 두유, 라지 사이즈로 3잔 추가해줘

단계 1: 요청 특성 분석
  - 복잡도: complex
  - 프라이버시: low
  - 응답 시간: fast
  - 민감 정보: 없음
  - 분석 근거: 주문 내용이 맞춤형 조합을 포함하고 있어 복잡하지만, 개인 정보는 포함되어 있지 않으며 즉각적인 응답이 필요하다.

단계 2: 서빙 선택
  - 선택된 서빙: OpenAI GPT-4o-mini
  - 선택 이유: 주문 내용이 맞춤형 조합을 포함하고 있어 복잡하고, 높은 품질의 응답이 필요하다.
  - 신뢰도: 0.90
  - 예상 비용: $0.000006

단계 3: 응답 생성

주 서비스 시도: OpenAI GPT-4o-mini
  성공: 응답이 유효하다

응답 생성 완료:
  - 사용된 서빙: cloud
  - 모델: gpt-4o-mini
  - 지연시간: 5224ms
  - 토큰: 1144개
  - 비용: $0.000343
  - 폴백 사용: 아니오

에이전트: {
  "complexity": "complex",
  "privacy_sensitivity": "low",
  "response_time_requirement": "fast",
  "contains_sensitive_data": false,
  "estimated_tokens": 23,
  "reasoning": "주문 내용이 구체적이고 맞춤형 조합을 포함하고 있어 복잡한 주문 요청으로 평가됨."
}

[cloud | 5224ms | $0.000343]



질문:  quit



대화를 종료한다.

총 비용: $0.001687
비용 절감: $0.000000


## 15. 성능 분석 및 리포트

In [17]:
print("\n" + "="*60)
print("성능 분석 리포트")
print("="*60)

# 1. 전체 통계
performance_summary = feedback_system.get_performance_summary()

print("\n1. 전체 통계")
print(f"  총 비용: ${performance_summary['total_cost']:.6f}")
print(f"  비용 절감: ${performance_summary['cost_savings']:.6f}")

# 2. Cloud 서비스 성능
cloud_perf = performance_summary['cloud']
print("\n2. Cloud 서비스 (OpenAI GPT-4o-mini)")
print(f"  - 요청 수: {cloud_perf['requests']}회")
print(f"  - 성공률: {cloud_perf['success_rate']:.1%}")
print(f"  - 평균 지연시간: {cloud_perf['avg_latency_ms']:.0f}ms")
print(f"  - 총 비용: ${cloud_perf['total_cost']:.6f}")

# 3. Local 서비스 성능
local_perf = performance_summary['local']
print("\n3. Local 서비스 (Ollama Llama3)")
print(f"  - 요청 수: {local_perf['requests']}회")
print(f"  - 성공률: {local_perf['success_rate']:.1%}")
print(f"  - 평균 지연시간: {local_perf['avg_latency_ms']:.0f}ms")
print(f"  - 총 비용: ${local_perf['total_cost']:.6f}")

# 4. 서비스 효율성 분석
print("\n4. 서비스 효율성 분석")
efficiency = feedback_system.analyze_service_efficiency()
for endpoint_id, eff in efficiency.items():
    endpoint = SERVING_ENDPOINTS[endpoint_id]
    print(f"\n  {endpoint.endpoint_name}:")
    print(f"    - 종합 점수: {eff['score']:.2f}")
    print(f"    - 비용 효율성: {eff['cost_efficiency']:.2f}")
    print(f"    - 속도 점수: {eff['speed_score']:.2f}")
    print(f"    - 신뢰도 점수: {eff['reliability_score']:.2f}")

# 5. 폴백 통계
print("\n5. 폴백 통계")
fallback_stats = recovery_system.get_fallback_stats()
print(f"  - 폴백 발생 횟수: {fallback_stats['count']}회")
if fallback_stats['most_common_reason']:
    print(f"  - 주요 원인: {fallback_stats['most_common_reason']}")

# 6. 최적화 제안
print("\n6. 최적화 제안")
recommendations = feedback_system.generate_optimization_recommendations()
for i, rec in enumerate(recommendations, 1):
    print(f"  {i}. {rec}")

# 7. 메모리 상태
print("\n7. 메모리 상태")
print(f"  - {memory.get_summary()}")

print("\n" + "="*60)
print("튜토리얼 4: Router LLM Serving Agent 완료")
print("="*60)


성능 분석 리포트

1. 전체 통계
  총 비용: $0.001687
  비용 절감: $0.000000

2. Cloud 서비스 (OpenAI GPT-4o-mini)
  - 요청 수: 7회
  - 성공률: 100.0%
  - 평균 지연시간: 2629ms
  - 총 비용: $0.001687

3. Local 서비스 (Ollama Llama3)
  - 요청 수: 0회
  - 성공률: 0.0%
  - 평균 지연시간: 0ms
  - 총 비용: $0.000000

4. 서비스 효율성 분석

  OpenAI GPT-4o-mini:
    - 종합 점수: 0.70
    - 비용 효율성: 1.00
    - 속도 점수: 0.00
    - 신뢰도 점수: 1.00

  Ollama exaone3.5:
    - 종합 점수: 0.00
    - 비용 효율성: 0.00
    - 속도 점수: 0.00
    - 신뢰도 점수: 0.00

5. 폴백 통계
  - 폴백 발생 횟수: 4회
  - 주요 원인: 'response'

6. 최적화 제안
  1. OpenAI GPT-4o-mini의 응답 속도가 느리다 (평균 2629ms). 더 빠른 서비스를 우선 사용하라.
  2. 로컬 서비스 사용률이 0.0%로 낮다. 간단한 질문에 로컬 서비스를 더 활용하여 비용을 절감하라.

7. 메모리 상태
  - 총 요청: 7회, 비용: $0.0017, 절감: $0.0000

튜토리얼 4: Router LLM Serving Agent 완료
